In [ ]:
import threading
import time

from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
import pandas as pd
import pytz
import numpy as np
import os
from sklearn import preprocessing
import re
import matplotlib
from matplotlib.patches import Polygon, Rectangle
matplotlib.use('Qt5Agg')
from datetime import timedelta
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from casadi import *
import calendar
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVR
import casadi as cd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, SGDRegressor


In [ ]:
training_csv_file_path = 'D:\\mlinternship\\iitgdata\\training_data_for_svm.csv'
df = pd.read_csv(training_csv_file_path)
df

In [ ]:
x = df.loc[:, 'CDH':'omega168']
x = x.to_numpy()
y = df['MW']
y = y.to_numpy()
y = y.reshape(-1,1)
scaler = StandardScaler()
x = scaler.fit_transform(x)
y = scaler.fit_transform(y)

In [ ]:
x

In [ ]:
y

SVM Part

In [ ]:
# C is inversely proportional to the number of misclassifications you allow
# epsilon is the allowed error in each instance

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)
y = y.reshape(len(y),)

In [ ]:
'''yhat = svr_model.predict(x)
plt.ion()
plt.figure(figsize=(12, 6))
plt.plot(training_df['Time'], yhat, marker='o', linestyle='-', color = 'g')
plt.plot(training_df['Time'], y, marker='x', linestyle='-', color = 'r')
plt.xlabel('Time')
plt.ylabel('Power Consumption Changes')

plt.grid(True)
plt.legend()
plt.show()'''

In [ ]:
C = [0.01, 0.1,1,5,10,15,50,100,1000]
gamma = [0.0001,0.001,0.01, 0.1,1,5,100]
epsilon = [0.05, 0.1, 0.2]
degree = [1,2,3,4,5,6]

results = {}
for d in degree:
    for e in epsilon:
        for c in C:
            for g in gamma:
                poly_svr = SVR(kernel='poly', degree=d,epsilon=e,C=c, gamma=g)
                poly_svr.fit(x, y)
                y_pred = poly_svr.predict(X_test)
                mse = mean_squared_error(y_test, y_pred)
                score  = poly_svr.score(X_test, y_test)
                hyperparameters = {'Degree': d, 'Epsilon': e, 'C': c, 'Gamma': g}
                result_values = {'MSE': mse, 'Score': score}
                results[str(hyperparameters)] = result_values
                print("Degree: " , d, "Epsilon: ",e,"C: ", c, "gamma: ", g, "MSE: ", mse, "score: ", score)



In [ ]:
stop

In [ ]:

'''C_values = [0.01, 0.1, 1, 5, 10, 15, 50, 100, 1000]
gamma_values = [5, 100]

def fit_svr_with_timeout(C, gamma):
    start_time = time.time()

    # Create SVR with specified parameters
    poly_svr = SVR(kernel='poly', C=C, gamma=gamma)

    # Define a function to be called by a timer
    def timeout_function():
        print(f"Timeout: C={C}, gamma={gamma}")
        poly_svr.fit_timed_out = True

    timer = threading.Timer(5, timeout_function)

    try:
        timer.start()
        poly_svr.fit(x, y)
    finally:
        timer.cancel()

    elapsed_time = time.time() - start_time
    if poly_svr.fit_timed_out:
        print(f"Skipped (elapsed time: {elapsed_time:.2f} seconds): C={C}, gamma={gamma}")
        return None

    # Make predictions on the test set
    y_pred = poly_svr.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    score = poly_svr.score(X_test, y_test)

    print(f"C: {C}, gamma: {gamma}, MSE: {mse}, score: {score}, elapsed time: {elapsed_time:.2f} seconds")


# Iterate over hyperparameter combinations
for i in C_values:
    for j in gamma_values:
        fit_svr_with_timeout(i, j)
'''

In [ ]:
param_grid = {'C':[0.01, 0.1,1,5,10,15,50,100,1000],
             'kernel':['linear'],
              'epsilon':[0.05, 0.1, 0.2]}

linear_svr_grid = SVR()
# Create a GridSearchCV object
linear_grid_search = GridSearchCV(linear_svr_grid,param_grid=param_grid)

# Fit the grid search to the data
linear_grid_search.fit(x, y)

# Get the best parameters and the best model
linear_best_params = linear_grid_search.best_params_

# Make predictions on the test set
linear_y_pred = linear_grid_search.predict(X_test)
mse = mean_squared_error(y_test, linear_y_pred)

print("Best Parameters:", linear_best_params)
print("Mean Squared Error on Test Set:", mse)
print("score of the model: ", linear_grid_search.score(X_test,y_test))

In [ ]:
param_grid = {'C':[0.01, 0.1,1,5,10,15,50,100,1000],
             'kernel':['rbf'],
              'gamma': [0.0001,0.001,0.01, 0.1,1,5,10,15,50]
              }

rbf_svr_grid = SVR()
# Create a GridSearchCV object
rbf_grid_search = GridSearchCV(rbf_svr_grid,param_grid=param_grid)

# Fit the grid search to the data
rbf_grid_search.fit(x, y)

# Get the best parameters and the best model
poly_best_params = rbf_grid_search.best_params_

# Make predictions on the test set
rbf_y_pred = rbf_grid_search.predict(X_test)
mse = mean_squared_error(y_test, rbf_y_pred)

print("Best Parameters:", poly_best_params)
print("Mean Squared Error on Test Set:", mse)
print("score of the model: ", rbf_grid_search.score(X_test,y_test))

In [ ]:
stop

In [ ]:
param_grid = {'C':[0.01,0.1,1,5,10,100,1000],
             'kernel':['poly'],
              'gamma': [0.001,0.01, 0.1,1,5],
              'degree':[1,2,3,4],
              'epsilon':[0.05, 0.1, 0.2]
              }

poly_svr_grid = SVR()
# Create a GridSearchCV object
poly_grid_search = RandomizedSearchCV(poly_svr_grid,param_distributions=param_grid,n_iter = 10 ,verbose= 5)

# Fit the grid search to the data
poly_grid_search.fit(x, y)

# Get the best parameters and the best model
poly_best_params = poly_grid_search.best_params_

# Make predictions on the test set
poly_y_pred = poly_grid_search.predict(X_test)
mse = mean_squared_error(y_test, poly_y_pred)

print("Best Parameters:", poly_best_params)
print("Mean Squared Error on Test Set:", mse)
print("score of the model: ", poly_grid_search.score(X_test,y_test))

In [ ]:
stop

In [ ]:
yhat = grid_search.predict(x)
plt.ion()


plt.figure(figsize=(12, 6))
plt.plot(training_df['Time'], yhat, marker='o', linestyle='-', color = 'g', label = 'predicted')
plt.plot(training_df['Time'], y, marker='x', linestyle='-', color = 'r', label = 'actual')
plt.xlabel('Time')
plt.ylabel('Power Consumption Changes')

plt.grid(True)
plt.legend()
plt.show()